In [2]:
import pandas as pd
import numpy as np
import json
import csv
import requests
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy import MetaData

In [3]:
def connect_db(database='dex_volume'):
    user='root'
    password='240699'
    host='localhost'
    database=database
    

    # Connect to the database
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    try:
        connection = engine.connect()
        return connection
    except Exception as e:
        raise e

In [4]:
response = requests.get('https://api.llama.fi/overview/dexs?excludeTotalDataChart=false&excludeTotalDataChartBreakdown=false&dataType=dailyVolume')
data = response.json() 

In [26]:
vol_historic = pd.DataFrame.from_dict(data['totalDataChart'], orient='columns')
vol_historic.columns = ['date', 'volume']
vol_historic.date = pd.to_datetime(vol_historic['date'],unit='s')


### Get Cumulative Volume and Pct change 1d, 7d, 30d, 90d, 180d & 1y


In [27]:
# Calculate percentage changes for different time intervals
vol_historic['1d_change'] = vol_historic['volume'].pct_change() * 100
vol_historic['7d_change'] = vol_historic['volume'].pct_change(periods=7) * 100
vol_historic['30d_change'] = vol_historic['volume'].pct_change(periods=30) * 100
vol_historic['90d_change'] = vol_historic['volume'].pct_change(periods=90) * 100
vol_historic['180d_change'] = vol_historic['volume'].pct_change(periods=180) * 100
vol_historic['1y_change'] = vol_historic['volume'].pct_change(periods=365) * 100

vol_historic['cumulative_volume'] = vol_historic['volume'].cumsum()



In [24]:
# Convert 'date' column to pandas datetime object
vol_historic['date'] = pd.to_datetime(vol_historic['date'], unit='s')

# Format the 'date' column in the desired date-only format
vol_historic['date'] = vol_historic['date'].dt.strftime('%d%m%y')

In [17]:
vol_historic

,date,volume,1d_change,7d_change,30d_change,90d_change,180d_change,1y_change,cumulative_volume
0,2019-10-11,9.435028e+05,NaN,NaN,NaN,NaN,NaN,NaN,9.435028e+05
1,2019-10-12,6.372715e+05,-32.456849,NaN,NaN,NaN,NaN,NaN,1.580774e+06
2,2019-10-13,5.939759e+05,-6.793902,NaN,NaN,NaN,NaN,NaN,2.174750e+06
3,2019-10-14,1.160122e+06,95.314610,NaN,NaN,NaN,NaN,NaN,3.334872e+06
4,2019-10-15,1.118992e+06,-3.545265,NaN,NaN,NaN,NaN,NaN,4.453864e+06
...,...,...,...,...,...,...,...,...,...
1538,2023-12-27,5.715523e+09,3.748501,14.357678,98.018180,198.170080,46.837278,398.293910,3.846770e+12
1539,2023-12-28,6.260207e+09,9.529912,22.127652,114.706245,281.296172,282.839723,316.689411,3.853030e+12
1540,2023-12-29,5.146664e+09,-17.787645,-25.674320,86.553891,321.838635,191.039088,295.016472,3.858177e+12
1541,2023-12-30,3.163717e+09,-38.528778,-25.735201,18.485785,89.579779,35.066681,200.866271,3.861341e+12


In [28]:
with connect_db() as connection:
    vol_historic.to_sql( 'dex_volumes_historic', con = connection, if_exists = 'replace' )

In [31]:
data['protocols']

[{'defillamaId': '2116',
  'name': '0x',
  'disabled': False,
  'displayName': '0x - RFQ',
  'module': '0x',
  'category': 'DEX Aggregator',
  'logo': 'https://icons.llamao.fi/icons/protocols/0x.png',
  'change_1d': None,
  'change_7d': None,
  'change_1m': None,
  'change_7dover7d': None,
  'change_30dover30d': None,
  'total24h': None,
  'total48hto24h': None,
  'total7d': None,
  'total30d': None,
  'total14dto7d': None,
  'total60dto30d': None,
  'total1y': None,
  'average1y': None,
  'totalAllTime': None,
  'breakdown24h': None,
  'chains': ['Ethereum', 'Polygon'],
  'protocolType': 'protocol',
  'methodologyURL': 'https://github.com/DefiLlama/dimension-adapters/blob/master/dexs/0x',
  'methodology': {},
  'latestFetchIsOk': True,
  'versionKey': '0x RFQ',
  'dailyVolume': None,
  'totalVolume7d': None,
  'totalVolume30d': None},
 {'defillamaId': '2345',
  'name': '10KSwap',
  'disabled': False,
  'displayName': '10KSwap',
  'module': '10kswap',
  'category': 'Dexes',
  'logo': '

In [33]:
vol_general = pd.DataFrame.from_dict(data['protocols'], orient='columns')
vol_general = vol_general[['defillamaId', 'name', 'disabled', 'displayName', 'module', 'category',
       'logo', 'change_1d', 'change_7d', 'change_1m', 'change_7dover7d',
       'change_30dover30d', 'total24h', 'total48hto24h', 'total7d', 'total30d',
       'total14dto7d', 'total60dto30d', 'totalAllTime',# 'breakdown24h','chains', 
                           'protocolType', 'methodologyURL', 
#                            'methodology',
       'latestFetchIsOk', 'versionKey', 'dailyVolume', 'parentProtocol']]


In [36]:
with connect_db() as connection: 
    vol_general.to_sql( 'dex_volume_general', con = connection, if_exists = 'replace')


In [38]:
data.keys()

dict_keys(['totalDataChart', 'totalDataChartBreakdown', 'protocols', 'allChains', 'chain', 'total24h', 'total48hto24h', 'total7d', 'total14dto7d', 'total60dto30d', 'total30d', 'total1y', 'average1y', 'change_1d', 'change_7d', 'change_1m', 'totalVolume7d', 'totalVolume30d', 'change_7dover7d', 'change_30dover30d', 'breakdown24h'])

### This one is for breakdown. each day : All Dex's Volume, Each


In [46]:
data['totalDataChartBreakdown']

[['1570752000', {'Bisq': 13942.623422953993, 'Uniswap V1': 929560.1686759855}],
 ['1570838400',
  {'Bisq': 267822.33333584055, 'Uniswap V1': 369449.18406718737}],
 ['1570924800', {'Bisq': 168462.5573356803, 'Uniswap V1': 425513.3585221346}],
 ['1571011200', {'Bisq': 373949.996690286, 'Uniswap V1': 786171.7472607518}],
 ['1571097600', {'Bisq': 454213.55668458476, 'Uniswap V1': 664778.7958460094}],
 ['1571184000', {'Bisq': 234763.3685736088, 'Uniswap V1': 1216313.13046972}],
 ['1571270400',
  {'Bisq': 142353.48312777607, 'Uniswap V1': 1140773.5578648439}],
 ['1571356800', {'Bisq': 515462.4594300941, 'Uniswap V1': 800401.0827349544}],
 ['1571443200', {'Bisq': 155470.7984998993, 'Uniswap V1': 370155.3896429694}],
 ['1571529600', {'Bisq': 122208.46886287187, 'Uniswap V1': 442230.7939487982}],
 ['1571616000', {'Bisq': 154575.78071164916, 'Uniswap V1': 761421.1488565959}],
 ['1571702400', {'Bisq': 73354.43396142854, 'Uniswap V1': 676933.8821930528}],
 ['1571788800', {'Bisq': 76691.35204042392

In [49]:
data['average1y']

239011291.28152373